# カルマンフィルタと粒子フィルタ

本notebookでは、カルマンフィルタと粒子フィルタについて解説する。ともに状態空間モデルのパラメータ推定法の一種である。  
参考文献: 
- カルマンフィルタ/粒子フィルタ
    - [基礎からわかる時系列分析(萩原ら, 2018)](https://gihyo.jp/book/2018/978-4-7741-9646-6)
    - [予測にいかす統計モデリングの基本(樋口, 2011)](https://www.kspub.co.jp/book/detail/1557956.html)
    - [カルマンフィルター(野村, 2016)](https://www.kyoritsu-pub.co.jp/bookdetail/9784320112537)
- 非線形カルマンフィルタ
    - [非線形カルマンフィルタ(片岡, 2011)](https://www.asakura.co.jp/books/isbn/978-4-254-20148-2/)
- 動的線形モデル
    - [Rによるベイジアン動的線形モデル(Petrisら, 2013)](https://www.amazon.co.jp/dp/4254127960/ref=cm_sw_em_r_mt_dp_U_O3NyDb5GKYQBX)

## 状態空間モデルの復習と詳細

### 概論

状態空間モデルは、状態$s$が時間に応じて変化し、その状態から観測値(データ)$y$が発生していると仮定するモデルである。$s$や$y$はベクトルでも良い。  
また、以下が仮定されている。
1. 各時点の状態$s_t$は、一つ前の時点の状態$s_{t-1}$のみと関係している
2. 各時点の観測値$y_t$は同一時点の状態のみと関係している

上記1.の性質は**マルコフ性**と呼ばれる。

数式で書くと、以下のように表せる。
$$s_t \sim F(s_{t-1})$$
$$y_t \sim H(s_{t})$$

ここで、$F$, $H$はそれぞれ何らかの確率分布である。それは正規分布のようにきれいに数式で表せるかもしれないし、きれいな数式では表せない分布かもしれない。とにかく、何らかの形をした確率分布である。

状態空間モデルをグラフィカルモデル(関係しあっている項目を矢印で表した図)で表すと、以下のようになる。

![グラフィカルモデル](figures/graphical_model_state_space.jpg)

### (Optional)状態空間モデルの同時確率

グラフィカルモデルからもわかるように、状態は一つ前の状態とだけ関係しており、観測値は同一時点の状態とだけ関係しており、他とは独立である。したがって、あるデータの系列$y_{1:T}$（すなわち$y_1, y_2, y_3, ..., y_T$）が得られたとき、そのようなデータが得られる確率は以下で得られる。
$$p(y_{1:T}) = p(s_0)\left(\prod_{k=1}^T p(s_k|s_{k-1})\right) \left(\prod_{k=1}^T p(y_k|s_k)\right)$$

ここで、
- $p(y_{1:T})$はデータがある特定の値の系列$y_{1:T}$を取る確率
- p(s_k|s_{k-1})は前時点の状態の値$s_{k-1}$が与えられたときに、今時点の状態がある値$s_k$を取る確率。確率分布$F(s_{k-1})$に従う。
- p(y_k|s_k)は今時点の状態の値$s_k$が与えられたときに、今時点の観測値がある値$y_k$を取る確率。確率分布$H(s_k)$に従う。

である。
ちなみに$\prod$は掛け合わせる記号であり、$\sum$の掛け算版である。

### 状態空間モデルの4タイプ

状態空間モデルは、大別して以下の4タイプに分けることができる。

1. <font color=blue>線形</font>・<font color=blue>ガウス</font>型
2. <font color=red>非線形</font>・<font color=blue>ガウス</font>型
3. <font color=blue>線形</font>・<font color=red>非ガウス</font>型
4. <font color=red>非線形</font>・<font color=red>非ガウス</font>型

適用できる各種手法については、
- **カルマンフィルタ**はタイプ1にのみ使用できる。
- **非線形カルマンフィルタ**はタイプ2に使用できる。
- **一般化動的線形モデル**はタイプ3に使用できる。
- **粒子フィルタ**と**MCMC**は全てのタイプに使用できる。

#### <font color=blue>線形</font>・<font color=blue>ガウス</font>型

<font color=blue>線形</font>・<font color=blue>ガウス</font>型は、$s_t$と$y_t$が以下の形で書けるタイプである。
$$s_t = A \cdot s_{t-1} + w_{t}$$
$$y_t = B \cdot s_{t} + v_{t}$$

ただし、$w_t$は状態遷移のバラツキ、$v_t$は観測誤差であり、それぞれ(多次元)正規分布に従うとする。すなわち、$W$,$V$を分散(または分散共分散行列)として、
$$w_{t} \sim N(0, W)$$
$$v_{t} \sim N(0, V)$$
と書ける。

$A$、$B$はそれぞれ何らかの形の行列である。$A$は状態の遷移(移り変わり)に関連しているので**遷移行列**、$B$は状態からの観測値が生成に関連しているため、**観測行列**と呼ばれる。

ちなみに、$s_t$、$y_t$は以下のようにも書ける。
$$s_t \sim N(A \cdot s_{t-1}, W)$$
$$y_t \sim N(B \cdot s_{t}, V)$$

<font color=blue>線形</font>・<font color=blue>ガウス</font>型の状態空間モデルのパラメータ推定方法は、以下が挙げられる。下にいくにつれて、計算量が大きくなる。
1. **カルマンフィルタ**で状態s$_t$を推定。遷移行列$A$、観測行列$B$、遷移のバラツキの分散$W$、観測誤差の分散$V$は固定、または最尤推定で推定。
2. **粒子フィルタ**で状態s$_t$を推定。遷移行列$A$、観測行列$B$、遷移のバラツキの分散$W$、観測誤差の分散$V$は固定、または最尤推定で推定。
3. **自己組織型粒子フィルタ**で状態s$_t$、遷移行列$A$、観測行列$B$、遷移のバラツキの分散$W$、観測誤差の分散$V$を推定。
4. **MCMC**で状態s$_t$、遷移行列$A$、観測行列$B$、遷移のバラツキの分散$W$、観測誤差の分散$V$を推定。

#### <font color=red>非線形</font>・<font color=blue>ガウス</font>型

<font color=red>非線形</font>・<font color=blue>ガウス</font>型は、$s_t$と$y_t$が以下の形で書けるタイプである。
$$s_t = f(s_{t-1}) + w_{t}$$
$$y_t = g(s_{t}) + v_{t}$$

ただし、先程同様、$w_t$は状態遷移のバラツキ、$v_t$は観測誤差であり、それぞれ(多次元)正規分布に従うとする。すなわち、$W$,$V$を分散(または分散共分散行列)として、
$$w_{t} \sim N(0, W)$$
$$v_{t} \sim N(0, V)$$
と書ける。

$f$、$g$はそれぞれ何らかの形の関数である。$f$は状態の遷移(移り変わり)に関連している関数、$g$は状態からの観測値が生成に関連している関数である。

ちなみに、$s_t$、$y_t$は以下のようにも書ける。
$$s_t \sim N\left(f(s_{t-1}), W \right)$$
$$y_t \sim N\left(g(s_{t}), V \right)$$

<font color=red>非線形</font>・<font color=blue>ガウス</font>型の状態空間モデルのパラメータ推定方法は、以下が挙げられる。下にいくにつれて、計算量が大きくなる。
1. **非線形カルマンフィルタ**で状態s$_t$を推定。関数$f$、関数$g$、遷移のバラツキの分散$W$、観測誤差の分散$V$は固定、または最尤推定で推定。
2. **粒子フィルタ**で状態s$_t$を推定。関数$f$、関数$g$、遷移のバラツキの分散$W$、観測誤差の分散$V$は固定、または最尤推定で推定。
3. **自己組織型粒子フィルタ**で状態s$_t$、関数$f$、関数$g$、遷移のバラツキの分散$W$、観測誤差の分散$V$を推定。
4. **MCMC**で状態s$_t$、関数$f$、関数$g$、遷移のバラツキの分散$W$、観測誤差の分散$V$を推定。

※非線形カルマンフィルタとしては、拡張カルマンフィルタ(EKF)、無香性カルマンフィルタ(UKF)、アンサンブルカルマンフィルタ(EnKF)、などが挙げられる。これらについてはここでは扱わない。[非線形カルマンフィルタ(片岡, 2011)](https://www.asakura.co.jp/books/isbn/978-4-254-20148-2/)を参照されたい。

#### <font color=blue>線形</font>・<font color=red>非ガウス</font>型

<font color=blue>線形</font>・<font color=red>非ガウス</font>型は、$s_t$と$y_t$が以下の形で書けるタイプである。
$$s_t = A \cdot s_{t-1} + w_{t}$$
$$y_t = B \cdot s_{t} + v_{t}$$

ただし、$w_t$は状態遷移のバラツキ、$v_t$は観測誤差であり、それぞれ何らかの正規分布以外の確率分布に従うとする(例えばポアソン分布など)。それらの確率分布をそれぞれ$X$, $Y$と置くと、数式では
$$w_{t} \sim X$$
$$v_{t} \sim Y$$
ということである。

<font color=blue>線形</font>・<font color=red>非ガウス</font>型の状態空間モデルのパラメータ推定方法は、以下が挙げられる。下にいくにつれて、計算量が大きくなる。
1. **一般化動的線形モデル**で状態s$_t$を推定。遷移行列$A$、観測行列$B$、遷移のバラツキの分布$X$、観測誤差の分布$Y$は固定、または最尤推定で推定。
2. **粒子フィルタ**で状態s$_t$を推定。遷移行列$A$、観測行列$B$、遷移のバラツキの分布$X$、観測誤差の分布$Y$は固定、または最尤推定で推定。
3. **自己組織型粒子フィルタ**で状態s$_t$、遷移行列$A$、観測行列$B$、遷移のバラツキの分布$X$、観測誤差の分布$Y$を推定。
4. **MCMC**で状態s$_t$、遷移行列$A$、観測行列$B$、遷移のバラツキの分布$X$、観測誤差の分布$Y$を推定。

一般化動的線形モデルについても、ここでは扱わない。[カルマンフィルター(野村, 2016)](https://www.kyoritsu-pub.co.jp/bookdetail/9784320112537)や[Rによるベイジアン動的線形モデル(Petrisら, 2013)](https://www.amazon.co.jp/dp/4254127960/ref=cm_sw_em_r_mt_dp_U_O3NyDb5GKYQBX)を参照されたい。

#### <font color=red>非線形</font>・<font color=red>非ガウス</font>型

<font color=red>非線形</font>・<font color=red>非ガウス</font>型は、$s_t$と$y_t$が以下の形で書けるタイプである。
$$s_t = f(s_{t-1}) + w_{t}$$
$$y_t = g(s_{t}) + v_{t}$$

ただし、$w_t$は状態遷移のバラツキ、$v_t$は観測誤差であり、それぞれ何らかの正規分布以外の確率分布に従うとする(例えばポアソン分布や指数分布など)。それらの確率分布をそれぞれ$X$, $Y$と置くと、数式では
$$w_{t} \sim X$$
$$v_{t} \sim Y$$
ということである。

<font color=red>非線形</font>・<font color=red>非ガウス</font>型の状態空間モデルのパラメータ推定方法は、以下が挙げられる。下にいくにつれて、計算量が大きくなる。
1. **粒子フィルタ**で状態s$_t$を推定。関数$f$、$g$、遷移のバラツキの分布$X$、観測誤差の分布$Y$は固定、または最尤推定で推定。
2. **自己組織型粒子フィルタ**で状態s$_t$、関数$f$、$g$、遷移のバラツキの分布$X$、観測誤差の分布$Y$を推定。
3. **MCMC**で$s_t$、関数$f$、$g$、遷移のバラツキの分布$X$、観測誤差の分布$Y$を推定。

### <font color=blue>線形</font>・<font color=blue>ガウス</font>型状態空間モデルによる、AR、MA、ARMA、ARIMA、SARIMAモデルの表現

ARモデルは、次のように表すことができる。

## フィルタリングとスムージング

カルマンフィルタ及び粒子フィルタでは、MCMCのように全てのパラメータを一気に推定するのではなく、各時点$t$の状態$s_t$について、別々にパラメータを推定する。別々に推定する際には、時点tまでに得られているデータだけを使用して状態を推定を推定する方法と、将来も含む全てのデータを使用して状態を推定する方法の2種類があり、前者を**フィルタリング**、後者を**スムージング(または平滑化)**と呼ぶ。すなわち、
- フィルタリング: 時点1からtのデータ$y_{1:t}$を用いて、状態$s_t$を推定する
- スムージング(または平滑化): 時点1からTのデータ$y_{1:T}$を用いて、状態$s_t$を推定する。ただし、$1 \leq t \leq T$

当然、リアルタイムな予測/推測システムを作ろうと思えば、フィルタリングしか使えない。スムージングは予測目的には使用できず、もっぱら、時系列の要素分解など、仕組みの解明や調査に使用される印象である。  
ここではフィルタリングを扱い、スムージングは扱わない。詳細は[カルマンフィルター(野村, 2016)](https://www.kyoritsu-pub.co.jp/bookdetail/9784320112537)や[基礎からわかる時系列分析(萩原ら, 2018)](https://gihyo.jp/book/2018/978-4-7741-9646-6)、または[予測にいかす統計モデリングの基本(樋口, 2011)](https://www.kspub.co.jp/book/detail/1557956.html)などを参照されたい。

### フィルタリング

フィルタリング、すなわち時点1からtのデータ$y_{1:t}$を用いて、状態$s_t$を推定することは、データ$y_{1:t}$が与えられたもとでの状態$s_t$の条件付き確率の分布を計算することと同じである。数式で書けば、以下の条件付き確率の分布が知りたい。
$$p(s_t|y_{1:t})$$

![フィルタリングの図](figures/filtering_figure.jpg)

これは式変形によって、以下のように計算できる。
$\begin{eqnarray}
p(s_t|y_{1:t}) &=& p(s_t|y_{1:t-1}, y_t)\\
&=& \frac{p(s_t, y_t|y_{1:t-1})}{p(y_t|y_{t:t-1})} \\
&=& \frac{p(y_t|s_t, y_{1:t-1})\cdot p(s_t|y_{1:t-1})}{p(y_t|y_{t:t-1})} \\
&=& \frac{p(y_t|s_t) \cdot p(s_t|y_{1:t-1})}{p(y_t|y_{t:t-1})} \\
&=& \frac{p(y_t|s_t) \cdot \int p(s_t|s_{t-1})p(s_{t-1}|y_{1:t-1})ds_{t-1}}{\int p(y_t|s_t)p(s_t|y_{1:t-1})ds_t} \\
&=& \frac{p(y_t|s_t) \cdot \int p(s_t|s_{t-1})p(s_{t-1}|y_{1:t-1})ds_{t-1}}{\int \left\{ p(y_t|s_t)\int p(s_t|s_{t-1})p(s_{t-1}|y_{1:t-1})ds_{t-1}\right\}ds_t}
\end{eqnarray}$

最後の式は、分母は、分子を全ての状態について足しあわせている形になっており、それによって、この条件付き確率$p(s_t|y_{1:t})$の和は1となることが保証されている。(当然といえば当然だが。)  
なので計算上は、色々な$s_t$についての分子が分かれば十分である。

上の形で計算した確率分布$p(s_t|y_{1:t})$は**フィルタリング分布**と呼ばれる。

ちなみに、上記4段目の式(下記に再掲)であるが、
- $p(y_t|s_t)$: データ$y_t$に対するパラメータ$s_t$の尤度
- $p(s_t|y_{1:t-1})$: データ$y_t$が得られる前の状態$s_{t}$の事前分布

と考えれば、新たに得られたデータ$y_t$を用いたパラメータ$s_t$のベイズ推定、と考えることもできる。(この考え方だと式1段目~3段目は不要)

$$p(s_t|y_{1:t}) = \frac{p(y_t|s_t) \cdot p(s_t|y_{1:t-1})}{p(y_t|y_{t:t-1})}$$

## 将来予測

将来予測は、ある時点tまでのデータ$y_{1:t}$を用いて、時点t+1以降の状態$s_{t+k}$を推定することだと言い換えられる(ただし$1 \leq k$)。  
これは、データ$y_{1:t}$が与えられたもとでの状態$s_{t+k}$の条件付き確率の分布を計算することと同じである。数式で書けば、以下の条件付き確率の分布が知りたい。
$$(s_{t+k}|y_{1:t})$$

これは式変形によって、以下のように計算できる。
$\begin{eqnarray}
p(s_{t+k}|y_{1:t}) &=& \int p(s_{t+k}|s_{t+k-1})p(s_{t+k-1}|y_{1:t})ds_{t+k-1}\\
&=& \int p(s_{t+k}|s_{t+k-1}) \left( \int p(s_{t+k-1}|s_{t+k-2})p(s_{t+k-2}|y_{1:t})ds_{t+k-2} \right)ds_{t+k-1}\\
&=& \cdots \\
&=& \int p(s_{t+k}|s_{t+k-1}) \left( \int p(s_{t+k-1}|s_{t+k-2}) \cdots \left( \int p(s_{t+1}|y_{1:t})ds_{t+1} \right) \cdots ds_{t+k-2} \right)ds_{t+k-1}\\
\end{eqnarray}$

すなわち、積分計算を行うことで1時点先の状態の確率分布を推定し、それをk回繰り返すことで、k時点先の状態$s_{t+k}$の確率分布を求める。

![予測の図](figures/prediction_state_space.jpg)

## カルマンフィルタ

カルマンフィルタはルドルフ・カルマンによって提唱された、<font color=blue>線形</font>・<font color=blue>ガウス</font>型の状態空間モデルにおいて、状態$s_t$についてフィルタリングを行うための方法である。

$$s_t = A \cdot s_{t-1} + w_{t}$$
$$y_t = B \cdot s_{t} + v_{t}$$

$$\boldsymbol{x}_{k} = F_{k}\boldsymbol{x}_{k-1}+ \boldsymbol{u}_{k}+G_{k}\boldsymbol{w}_{k}$$

## 粒子フィルタ